We'll start off by setting the seed for reproducibility.

In [1]:
import sys
import torch
import random
import numpy as np

seed = 26

In [2]:
import sys
if 'google.colab' in sys.modules:  # If in Google Colab environment
    # Installing requisite packages
    !pip install datasets transformers evaluate
    !pip install accelerate -U
    !pip install -U sentence-transformers

    # Mount google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

    # Change working directory to desired folder
    %cd /content/drive/MyDrive/HS_23_Msc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.4 MB/

# Reading data

In [3]:
from transformers import pipeline
import pandas as pd
import torch

In [6]:
# Reading in the .csv data
dat = pd.read_csv('wordsOnly.csv')
dat # Inspecting the data

,word
0,Klimagesetz
1,Teuer
2,Stromteuerung
3,Hohe Kosten Stromleitungen
4,PV privat Unsinn
...,...
6672,Verbot fossiler Energie
6673,Unterstützung für Umstellung
6674,Schutz für Generationen
6675,Lösung von Ausland


# Extracting features

In [7]:
# Setting device to mps
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [8]:
# Initializing feature extractor
model_ckpt = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
feature_extractor = pipeline(
    'feature-extraction', model=model_ckpt, tokenizer=model_ckpt, device=device,
    framework='pt'
)

# Extracting features
texts = dat['word'].tolist()
features = [feature_extractor(text, return_tensors='pt')[0][0].numpy() for text in texts]
features = pd.DataFrame(features)
features

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,374,375,376,377,378,379,380,381,382,383
0,0.022616,0.415679,0.131571,0.320233,0.477956,0.060180,0.026229,-0.137214,-0.032047,0.342856,...,0.087627,-0.028879,0.130957,-0.066824,-0.387957,0.116138,-0.240112,-0.287597,-0.305492,0.033682
1,0.116735,0.307324,-0.068396,-0.248067,0.116704,-0.071267,0.342812,0.516092,0.175445,-0.018673,...,0.126176,-0.088379,0.028152,-0.058135,-0.285814,0.191686,0.137779,-0.065865,-0.063493,0.294646
2,-0.094757,0.562128,-0.044198,0.292374,0.324770,-0.193191,0.242892,0.277727,0.019026,0.093243,...,0.393377,-0.144331,0.088521,-0.083992,-0.406141,-0.099503,0.053925,0.101875,-0.094818,0.038419
3,-0.099935,0.261219,0.101228,0.162826,0.106694,-0.241507,0.143845,0.250619,0.034052,-0.024762,...,0.330932,-0.185408,-0.162031,-0.121693,-0.459234,-0.148149,0.038668,-0.021917,-0.079975,-0.328134
4,0.061350,0.271958,-0.092156,-0.227568,0.200132,-0.261179,-0.037341,0.346176,0.020389,0.113451,...,0.293944,-0.238364,-0.001133,0.073183,-0.340103,0.010322,0.184412,0.045912,-0.194272,0.166380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6672,-0.095826,0.924980,0.103206,0.182592,0.323248,-0.070965,-0.030580,0.204370,-0.046385,-0.068088,...,0.448530,0.060498,-0.251323,-0.153682,-0.807420,0.109732,0.353253,-0.587074,0.209011,-0.068146
6673,-0.002519,-0.043709,0.013033,0.018567,0.149138,0.097515,0.052506,-0.052883,-0.004502,0.268658,...,0.303804,-0.040584,-0.100940,0.325128,-0.127305,0.185159,0.287645,0.117083,-0.058757,0.018011
6674,0.000880,0.668269,-0.564156,-0.288791,0.345920,0.302440,0.284548,0.169501,0.071667,0.338745,...,0.173424,0.085148,0.049628,0.115981,-0.192213,0.277566,0.294779,-0.284527,0.092304,0.201903
6675,0.153483,-0.009160,0.092145,-0.157207,0.238008,-0.327515,0.098834,0.111302,-0.079096,0.265128,...,0.149119,-0.129748,-0.116736,-0.008131,-0.050811,0.265781,-0.091158,0.026879,0.060526,0.025057


In [9]:
features.to_csv('features_word.csv')